# Analysis of Variance - ANOVA

---

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.power import TTestIndPower

plt.style.use("fivethirtyeight")
%matplotlib inline

---

<a id="three-means-anova"></a>
## ANOVA

Per lavorare su gruppi di **tre o più** medie, non possiamo più usare la *t statistic*, ma dobbiamo ripiegare su un nuovo test chiamato **analysis of variance (ANOVA)** e una nuova metrica chiamata **F statistic**.

L'*anova* risulta utile per studiare le medie dei sotto-gruppi ricavati da una specifica variabile categorica (a più di due valori). La tipica ipotesi nulla afferma che tali medie sono coincidenti, mentre la tipica ipotesi alternativa afferma che *almeno* una coppia di esse è diversa dall'altra, ma non specifica quali.

In *anova* la *test statistic*, cioè la *f statistic*, è definita come il rapporto fra la variabilità media *fra* gruppi (**between groups variability**) e la variabilità *all'interno* dei gruppi (**within groups variability**).

La **F Distribution** è una distribuzione *right skewed* ed è sempre positiva. Al contrario della *t distribution*, qui avremo due parametri di *degrees of freedom*, uno legato al *group* e uno al *non-group*. Per poter rigettare $H_0$, abbiamo bisogno di un piccolo *p-value* e quindi di una grande *f-statistic*, che si può ottenere solo se la variabilità fra le *sample means* dei gruppi (*between*) è molto più grande della variabilità intrinseca fra i sample stessi (*within*).

Descriviamo il concetto di **variability partitioning**: dato un dataset composto da un certo numero di attributi e da una variabile categorica, la variabilità totale di uno di questi attributi può essere decomposta nella variabilità attribuibile alla variabile categorica (che rappresenta la *between group variability*) e la variabilità attribuibile a tutti gli altri fattori (che rappresenta la *within group variability*).

Il processo dell'*anova*, che porta al calcolo dell'*f-statistic*, può essere riassunto in una tabella a doppia entrata. Abbiamo solo due righe, etichettate con *Group* (che fa riferimento alla *Between Group Variability*) ed *Error* (che fa riferimento alla *Within Group Variability*). Nelle colonne, invece, troviamo *Df*, *Sum Sq*, *Mean Sq*, *F value* e *Pr(>F)*. Esiste anche una terza riga, che mostra semplicemente i valori *totali* contenuti nelle diverse colonne.

Per la colonna *Df*, possiamo calcolare direttamente il valore totale come il *sample size* meno uno, mentre i *df* della riga *Group* coincidono con il numero di gruppi meno uno. I gradi di libertà relativi alla riga *Error*, infine, si possono ricavare come la differenza fra il totale e i *df* dei gruppi.

$$\large df_T=n-1$$

$$\large df_G=k-1$$

$$\large df_E=df_T-df_G$$

Il totale della colonna *Sum Sq* è la **sum of squares total (SST)**, che evidenzia la variabilità totale della *response variable* (non la categorica). Si calcola in maniera molto simile alla *varianza*, con la differenza che non è scalata per il *sample size*.

$$\large SST=\sum_{i=1}^{n}\left( y_i-\bar{y} \right)^2$$

L'incrocio fra *Group* e *Sum Sq* ci dà la **sum of squares groups (SSG)**, che misura la variabilità della response variable *spiegata* dall'*explanatory variable* (la categorica). Si calcola come la deviazione quadrata delle medie dei gruppi rispetto alla media totale, pesata per il corrispondente *sample size* (cioè la cardinalità del gruppo, $\large n_j$).

$$\large SSG=\sum_{i=1}^{k}n_i(\bar{y}_i-\bar{y})^2$$

La **sum of squares error (SSE)**, invece, si calcola semplicemente come la differenza fra la variabilità totale e la variabilità *between groups*. Si tratta della *unexplained variability*, cioè di quella variabilità nella *response* dovuta ad altri fattori, non all'*explanatory variable*.

$$\large SSE=SST-SSG$$

Nella colonna *Mean Sq* troviamo la variabilità media, scomposta per *group* e *non-group*. Si può calcolare dividendo i risultati delle *sum of squares* per i corrispondenti *degrees of freedom*.

$$\large MSG=\frac{SSG}{df_G}$$

$$\large MSE=\frac{SSE}{df_E}$$

E' sulla base di questi valori che sarà calcolata l'*f statistic*, definita come il rapporto fra le due variabilità medie:

$$\large F=\frac{MSG}{MSE}$$

Con l'*f statistic* a disposizione, è possibile calcolare regolarmente il *p-value* e terminare il processo inferenziale. Per individuare la *f-distribution* di riferimento, i *df* da specificare sono nell'ordine $df_G$ e $df_E$. Nello specifico, se il *p-value* è più piccolo di $\alpha$, concludiamo che i dati suggeriscono che almeno una coppia di medie, appartenenti ai gruppi analizzati, presenta valori diversi tra loro.

<br>

Le condizioni per applicare l'*anova* sono le seguenti:
- **Independence within groups**: le osservazioni campionate devono essere indipendenti. Si verifica se applichiamo il *random sampling/assignment*, o se il *sample size* di ciascun gruppo è inferiore al 10% della corrispondente popolazione.


- **Independence between groups**: i gruppi devono essere indipendenti fra loro, cioè non paired. Se così non è, bisogna ripiegare sul **Repeated Measures Anova**.


- **Approximate normality**: le distribuzioni di ciascun gruppo dovrebbero essere quasi normali.


- **Equal variance**: la distribuzione della *response variable* nei vari gruppi deve avere più o meno la stessa variabilità. Si dice che i gruppi sono *homoscedastic* rispetto alla *response variable*.

---

Più in breve, il **One-way ANOVA** è un test che ha come ipotesi nulla di riferimento l'uguaglianza fra $k$ diverse medie e che prova a dimostrarne la validità confrontando la variabilità **between groups** con la variabilità **within groups**.

In generale, quando la variabilità *withing groups* è molto limitata, anche leggere variazioni di media fra i gruppi permettono di rigettare l'ipotesi nulla. Quando, invece, la variabilità *withing groups* è molto grande per ciascun gruppo, è più difficile ovviamente rigettare l'ipotesi nulla, anche se le medie dei gruppi sono nettamente diverse.

Specifichiamo che per il *one-way ANOVA test* bisogna assumere che le varianze delle popolazioni siano coincidenti, un po' come già visto per la *pooled variance procedure* nell'inferenza fra due medie:

$$\large \sigma_1^2=\dots=\sigma_k^2=\sigma^2$$

---

**[Esempio]** Studiare l'influenza dei vari dosaggi rispetto ai valori misurati di libido.

In [2]:
df = pd.DataFrame({"person": np.arange(1, 16),
                   "dose": [1,1,1,1,1,2,2,2,2,2,3,3,3,3,3],
                   "libido": [3,2,1,1,4,5,2,4,2,3,7,4,5,3,6]})
df["dose"].replace({1: "placebo", 2: "low", 3: "high"}, inplace = True)
df.head()

,person,dose,libido
0,1,placebo,3
1,2,placebo,2
2,3,placebo,1
3,4,placebo,1
4,5,placebo,4


_Calcolo rapido dell'one way anova test con scipy, per ottenere subito la f-statistic* ed il p-value._

In [3]:
F, p_value = stats.f_oneway(
    df["libido"][df["dose"] == "high"],
    df["libido"][df["dose"] == "low"],
    df["libido"][df["dose"] == "placebo"])

F, p_value

(5.11864406779661, 0.024694289538222603)

_Costruzione dell'intera anova table (attenzione alla "C" nell'OLS)_

In [4]:
model = smf.ols("libido ~ C(dose)", data = df).fit()

anova_table = sm.stats.anova_lm(model, typ = 2)
anova_table

,sum_sq,df,F,PR(>F)
C(dose),20.133333,2.0,5.118644,0.024694
Residual,23.600000,12.0,NaN,NaN


_Shapiro Test per verificare che i residui seguano una normale_

In [5]:
stats.shapiro(model.resid)

(0.9166916012763977, 0.17146942019462585)

_Il test NON è significativo, quindi i residui seguono andamento normale._

_Levene Test per verificare che i gruppi abbiano varianze uguali_

In [6]:
stats.levene(
    df["libido"][df["dose"] == "high"],
    df["libido"][df["dose"] == "low"],
    df["libido"][df["dose"] == "placebo"])

LeveneResult(statistic=0.11764705882352934, pvalue=0.8900225182757423)

_Il test NON è significativo, quindi i gruppi presentano una differenza in variabilità non significativa, e quindi hanno più o meno la stessa varianza._

---